# Setting

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

import torch
from datasets import Dataset, DatasetDict
from transformers import AdamW, EarlyStoppingCallback
from transformers import get_linear_schedule_with_warmup
from transformers import BertForSequenceClassification, BertTokenizer
from transformers import TrainingArguments, Trainer

import wandb

/home/xogns5037/.conda/envs/sparqlgen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wandb.init(project='chat_clf_bert_finetuning', name='run1')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: oiehhun (oiehhun-yonsei-university). Use `wandb login --relogin` to force relogin


# Data Load

데이터 출처 : https://github.com/songys/Chatbot_data \
데이터 설명 : 11,876개의 한글 대화 문답으로 되어 있는 인공데이터로, 일상 대화, 이별과 관련된 대화, 긍정적인 사랑에 대한 대화가 각각 0, 1, 2로 라벨링

In [2]:
chat_data = pd.read_csv('./K-CAT/lth/data/ChatbotData.csv',encoding="utf-8")
chat_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [3]:
chat_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


# Data preprocessing

- 데이터셋은 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2로 레이블링 되어 있음
- 일상 대화인지 이별 대화인지 사랑 대화인지 분류하는 문제를 풀기 위해 이별(0)/사랑(1) 레이블을 1로 통합
- 일상 대화(0), 연애 대화(1)

In [4]:
# 이별, 사랑 label을 1로 통합
chat_data.loc[(chat_data['label'] == 2), 'label'] = 1

In [5]:
chat_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,1
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,1
11820,흑기사 해주는 짝남.,설렜겠어요.,1
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,1


In [6]:
# train, test 데이터셋 셔플 및 분리
chat_data_suffled = chat_data.sample(frac=1).reset_index(drop=True)
train = chat_data_suffled[:9000]
test = chat_data_suffled[9000:]
print(train.shape, test.shape)

(9000, 3) (2823, 3)


# Model Load

In [8]:
# BERT 모델 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

/home/xogns5037/.conda/envs/sparqlgen/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# BERT 모델 불러오기
tokenizer = BertTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1', num_labels=2)

# Tokenizer

In [9]:
def tokenize_function(data):
    return tokenizer(
        data['Q'], data['A'],       # 대화 내용(Q : A)
        add_special_tokens=True,    # [CLS] Q [SEP] A [SEP]
        max_length=256,             # 문장 최대 길이
        truncation=True,            # 문장이 max_length보다 길면 자름
        padding='max_length'        # 문장이 max_length보다 짧으면 padding
    )

In [10]:
# tokenize 예시 결과
q_text = '안녕하세요?'
a_text = '반갑습니다!'

tokenized_output = tokenizer(
    q_text, a_text,
    add_special_tokens=True,
    max_length=32,
    truncation=True,
    padding='max_length',
)

print(f"Q: {q_text}")
print(f"A: {a_text}")
print(f"input_ids: {tokenized_output['input_ids']}") # tokenized된 문장을 숫자로 표현한 것(id는 vocab에 있는 단어의 index)
print(f"token_type_ids: {tokenized_output['token_type_ids']}") # 문장 구분을 위한 token_type_ids
print(f"attention_mask: {tokenized_output['attention_mask']}") # 실제 의미가 있는 token(모델이 참조해야할 부분)은 1, padding(참조하지 않아도 되는 부분)은 0
print(f"tokens: {tokenizer.convert_ids_to_tokens(tokenized_output['input_ids'])}") # input_ids를 다시 토큰화한 결과 : [CLS] 안녕하세요? [SEP] 반갑습니다! [SEP]

Q: 안녕하세요?
A: 반갑습니다!
input_ids: [101, 1463, 30006, 30021, 29992, 30010, 30025, 30005, 30006, 29997, 30009, 29999, 30013, 1029, 102, 1460, 30006, 30021, 29991, 30006, 30024, 29997, 30017, 30024, 29992, 30019, 29993, 30006, 999, 102, 0, 0]
token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
tokens: ['[CLS]', 'ᄋ', '##ᅡ', '##ᆫ', '##ᄂ', '##ᅧ', '##ᆼ', '##ᄒ', '##ᅡ', '##ᄉ', '##ᅦ', '##ᄋ', '##ᅭ', '?', '[SEP]', 'ᄇ', '##ᅡ', '##ᆫ', '##ᄀ', '##ᅡ', '##ᆸ', '##ᄉ', '##ᅳ', '##ᆸ', '##ᄂ', '##ᅵ', '##ᄃ', '##ᅡ', '!', '[SEP]', '[PAD]', '[PAD]']


# Dataset

In [11]:
# 검증(Vaildation) 데이터셋 분리
train_data, valid_data = train_test_split(train, test_size=0.1, random_state=42)
print(train_data.shape, valid_data.shape)

(8100, 3) (900, 3)


In [12]:
# Dataset 생성
train_dataset = Dataset.from_pandas(train_data) # pandas DataFrame -> Hugging Face Dataset 형식으로 변환
valid_dataset = Dataset.from_pandas(valid_data)
test_dataset = Dataset.from_pandas(test)

datasets = DatasetDict({'train': train_dataset, 'valid': valid_dataset, 'test': test_dataset}) # train, valid, test 데이터셋을 묶어서 저장
tokenized_datasets = datasets.map(tokenize_function, batched=True) # train, vaild, test 데이터셋에 tokenize_function 적용

Map: 100%|██████████| 2823/2823 [00:01<00:00, 2647.91 examples/s]


In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Q', 'A', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8100
    })
    valid: Dataset({
        features: ['Q', 'A', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 900
    })
    test: Dataset({
        features: ['Q', 'A', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2823
    })
})

# Train

In [14]:
# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir='./K-CAT/lth/model_save',        # 학습 결과 저장 경로
    report_to='wandb',                          # wandb 사용
    num_train_epochs=15,                        # 학습 epoch 설정
    per_device_train_batch_size=32,             # train batch_size 설정
    per_device_eval_batch_size=32,              # test batch_size 설정
    logging_dir='./K-CAT/lth/model_save/logs',  # 학습 log 저장 경로
    logging_steps=100,                          # 학습 log 기록 단위
    save_total_limit=2,                         # 학습 결과 저장 최대 개수
    evaluation_strategy="epoch",                # 매 epoch마다 평가 실행
    save_strategy="epoch",                      # 매 epoch마다 모델 저장
    load_best_model_at_end=True,                # 가장 성능이 좋은 모델을 마지막에 load
)

# 최적화 알고리즘(optimizer) 설정
optimizer = AdamW(model.parameters(), lr=2e-5)

# 스케줄러(scheduler) 설정
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(tokenized_datasets['train']) * training_args.num_train_epochs
)

/home/xogns5037/.conda/envs/sparqlgen/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
# 성능 평가 지표 설정(binary classification)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [16]:
# Trainer 생성
trainer = Trainer(
    model=model, 
    tokenizer=tokenizer,
    optimizers=(optimizer, scheduler),
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['valid'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [17]:
# 모델 학습
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.547100,0.454222,0.790000,0.806154,0.823899,0.789157
2,0.425900,0.436502,0.793333,0.791011,0.897959,0.706827
3,0.383200,0.398262,0.833333,0.850895,0.842520,0.859438
4,0.333600,0.439206,0.831111,0.834061,0.913876,0.767068
5,0.258500,0.418377,0.837778,0.845992,0.891111,0.805221
6,0.242200,0.433498,0.838889,0.848801,0.882863,0.817269
7,0.203300,0.447833,0.840000,0.856000,0.852590,0.859438
8,0.169300,0.482568,0.846667,0.860324,0.867347,0.853414


TrainOutput(global_step=2032, training_loss=0.32180882157303214, metrics={'train_runtime': 1409.1302, 'train_samples_per_second': 86.223, 'train_steps_per_second': 2.704, 'total_flos': 8524798193664000.0, 'train_loss': 0.32180882157303214, 'epoch': 8.0})

# Test

In [18]:
# 테스트(Test) 데이터셋 평가
trainer.evaluate(tokenized_datasets['test'])

{'eval_loss': 0.4072250425815582,
 'eval_accuracy': 0.824300389656394,
 'eval_f1': 0.8411274823830878,
 'eval_precision': 0.8357733927434755,
 'eval_recall': 0.8465506125080593,
 'eval_runtime': 20.9911,
 'eval_samples_per_second': 134.486,
 'eval_steps_per_second': 4.24,
 'epoch': 8.0}

In [19]:
# 실제 대화 테스트
def predict(q, a):
    model.eval()
    tokenized_sent = tokenizer(q, a, add_special_tokens=True, return_tensors='pt')
    tokenized_sent.to('cuda:0')
    
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
            token_type_ids=tokenized_sent["token_type_ids"]
            )
        
    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1)  
    
    if result == 0:
        return '일상 대화 🤖'
    elif result == 1:
        return '연애 대화 🥰'

for idx in range(1, 6):
    q = test_dataset[idx]['Q']
    a = test_dataset[idx]['A']
    label = test_dataset[idx]['label']
    print(f' Q: {q}\n A: {a}\n label: {label}\n pred: {predict(q, a)}\n')

 Q: 사랑의 유효기간
 A: 이별의 유통기한이 없는 것처럼 사랑의 유효기간도 없어요.
 label: 1
 pred: 연애 대화 🥰

 Q: 술취해서 전화했어
 A: 후회하지 않을까요.
 label: 1
 pred: 일상 대화 🤖

 Q: 좋아하는 사람이랑 종교가 다른데 괜찮을까?
 A: 종교가 큰 문제가 되기도 하죠.
 label: 1
 pred: 연애 대화 🥰

 Q: 이별후 너무 외로워ㅠ
 A: 이별의 빈자리가 느껴지니까요.
 label: 1
 pred: 연애 대화 🥰

 Q: 나 미팅한다!
 A: 성공을 기원합니다.
 label: 0
 pred: 일상 대화 🤖



wandb: 🚀 View run run1 at: https://wandb.ai/oiehhun-yonsei-university/chat_clf_bert_finetuning/runs/47zph67z
wandb: Find logs at: wandb/run-20250130_115005-47zph67z/logs
